In [117]:
import pandas as pd

In [118]:
df_raw = pd.read_excel('006__Impulse dataset.xlsx', None)

try:
    df.drop(df.index, inplace=True)
except NameError:
    pass

In [119]:
# список листов книги эксель
sheets = list(df_raw.keys())
for k, v in enumerate(sheets):
    print(k, v)

0 Основная комната – Зрители
1 QA – Зрители
2 DA – Зрители
3 Маркетинг – зрители
4 Дизайн – зрители
5 Python – зрители
6 Java – Зрители
7 Веб – зрители


In [120]:
# список нужных колонок листа
columns = ['Дата', 'Имя участника',
           'На вебинаре с', 'Досмотрел до', 'Интервалы', 'присутствия', 'Продолжительность',
           'Устройство', 'Клик по кнопке',
           'Город', 'Регион', 'Ссылка входа', 'Переход с']

mapping = {'Дата': 'date',
           'Имя участника': 'name',
           'На вебинаре с': 'webinar_from',
           'Досмотрел до': 'webinar_to',
           'Интервалы': 'interval_from',
           'присутствия': 'interval_to',
           'Продолжительность': 'duration',
           'Устройство': 'device',
           'Клик по кнопке': 'click_to',
           'Город': 'city',
           'Регион': 'region',
           'Ссылка входа': 'entrance_link',
           'Переход с': 'source_link'}

In [121]:
def dataframe_processing(_df_raw_,
                         _sheet_index_,
                         _columns_):
    cdf = pd.DataFrame(_df_raw_[sheets[_sheet_index_]][_columns_])

    cdf = cdf.rename(columns=mapping)

    cdf.loc[cdf['name'].notnull(), cdf.columns != 'name'] = \
        cdf.loc[cdf['name'].notnull(), cdf.columns != 'name'].fillna("____")
    cdf = cdf.ffill(axis=0)
    cdf['click_to'] = cdf['click_to'].str.split(', ')
    cdf = cdf.explode('click_to')
    cdf = cdf[cdf['webinar_to'] != "____"].dropna()

    cdf[['Button_Name', 'Button_Time']] = cdf['click_to'].str.split(' в ', expand=True)
    cdf = cdf.replace({'____': '', None: ''})

    cdf['date'] = pd.to_datetime(cdf['date'])

    try:
        cdf['webinar_from'] = pd.to_datetime(cdf['webinar_from'], format='%H:%M').dt.time
        cdf['webinar_to'] = pd.to_datetime(cdf['webinar_to'], format='%H:%M').dt.time
    except ValueError:
        cdf['webinar_from'] = pd.to_datetime(cdf['webinar_from'], format='%H:%M:%S').dt.time
        cdf['webinar_to'] = pd.to_datetime(cdf['webinar_to'], format='%H:%M:%S').dt.time

    cdf['interval_from'] = pd.to_datetime(cdf['interval_from'], format='%H:%M:%S').dt.time
    cdf['interval_to'] = pd.to_datetime(cdf['interval_to'], format='%H:%M:%S').dt.time

    from datetime import datetime
    cdf['Button_Time'] = cdf['Button_Time'].apply(
        lambda x: datetime.strptime(str(x), '%H:%M').time() if x != '' else '')

    return cdf

In [124]:
current = dataframe_processing(df_raw, 0, columns)
current

,date,name,webinar_from,webinar_to,interval_from,interval_to,duration,device,click_to,city,region,entrance_link,source_link,Button_Name,Button_Time
0,2023-01-14 11:56:00,Коротаев Павел,11:56:00,12:35:00,11:56:57,12:35:42,38.750000,ПК,«Python-разработчик» в 12:35,Липецк,Липецкая область,https://start.bizon365.ru/room/81268/impulse_g...,https://www.google.com/,«Python-разработчик»,12:35:00
1,2023-01-14 11:56:00,Роман Орлов,11:56:00,15:59:00,11:56:57,15:59:28,242.516667,ПК,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,
2,2023-01-14 11:56:00,Борисыч,11:56:00,15:58:00,11:56:57,12:45:42,48.750000,моб,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,
3,2023-01-14 11:56:00,Борисыч,11:56:00,15:58:00,12:47:52,12:47:57,0.083333,моб,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,
4,2023-01-14 11:56:00,Борисыч,11:56:00,15:58:00,12:50:47,12:50:52,0.083333,моб,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5496,2023-01-14 11:56:00,Анатолий+,15:50:00,15:52:00,15:50:48,15:52:23,1.583333,ПК,,Набережные Челны,Татарстан,https://start.bizon365.ru/room/81268/impulse_g...,https://click.mail.ru/,,
5497,2023-01-14 11:56:00,Дарья+,15:51:00,15:54:00,15:51:18,15:54:03,2.750000,моб,,Саратов,Саратовская область,https://start.bizon365.ru/room/81268/impulse_g...,,,
5498,2023-01-14 11:56:00,Alina,15:54:00,15:55:00,15:54:08,15:55:33,1.416667,моб,,Тбилиси,Тбилиси,https://start.bizon365.ru/room/81268/impulse_g...,,,
5499,2023-01-14 11:56:00,тест тестович,15:54:00,15:58:00,15:54:38,15:58:48,4.166667,ПК,,Батуми,Одесская область,https://start.bizon365.ru/room/81268/impulse_g...,,,


In [157]:
min_webinar = current.webinar_from.min()
max_webinar = current.webinar_to.max()
webinar_range = pd.date_range(str(min_webinar), str(max_webinar), freq="1min").time

webinar_range[0]

datetime.time(11, 56)

In [156]:
result = pd.DataFrame(columns=['_in_', '_out_',
                               'qa',
                               'da',
                               'marketing',
                               'design',
                               'python',
                               'java',
                               'web'],
                      index=webinar_range).reset_index()
result.rename(columns={'index': 'time'}).head(3)

,time,_in_,_out_,qa,da,marketing,design,python,java,web
0,11:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
lag = 3

In [158]:
current.head(3)

,date,name,webinar_from,webinar_to,interval_from,interval_to,duration,device,click_to,city,region,entrance_link,source_link,Button_Name,Button_Time
0,2023-01-14 11:56:00,Коротаев Павел,11:56:00,12:35:00,11:56:57,12:35:42,38.750000,ПК,«Python-разработчик» в 12:35,Липецк,Липецкая область,https://start.bizon365.ru/room/81268/impulse_g...,https://www.google.com/,«Python-разработчик»,12:35:00
1,2023-01-14 11:56:00,Роман Орлов,11:56:00,15:59:00,11:56:57,15:59:28,242.516667,ПК,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,
2,2023-01-14 11:56:00,Борисыч,11:56:00,15:58:00,11:56:57,12:45:42,48.750000,моб,,Москва,Москва,https://start.bizon365.ru/room/81268/impulse_g...,,,


In [199]:
current.sketch.howto("loc 'name' if 'webinar_from'>11:56:00 and 'webinar_to'<11:57:00 and 'duration'>'lag'")

In [198]:

lag = 0

# Count the number of rows where webinar_from is greater than 11:56:00, webinar_to is less than 11:57:00, and duration is greater than lag
print([webinar_range[0], webinar_range[1]])
print(current[(current['webinar_from'] >= webinar_range[0]) &
              (current['webinar_from'] <= webinar_range[1])])


[datetime.time(11, 56), datetime.time(11, 57)]
                    date              name webinar_from webinar_to  \
0    2023-01-14 11:56:00    Коротаев Павел     11:56:00   12:35:00   
1    2023-01-14 11:56:00       Роман Орлов     11:56:00   15:59:00   
2    2023-01-14 11:56:00           Борисыч     11:56:00   15:58:00   
3    2023-01-14 11:56:00           Борисыч     11:56:00   15:58:00   
4    2023-01-14 11:56:00           Борисыч     11:56:00   15:58:00   
...                  ...               ...          ...        ...   
1598 2023-01-14 11:56:00              Саид     11:57:00   12:15:00   
1599 2023-01-14 11:56:00              Саид     11:57:00   12:15:00   
1601 2023-01-14 11:56:00  Копаев Станислав     11:57:00   12:23:00   
1602 2023-01-14 11:56:00           Дмитрий     11:57:00   15:30:00   
1603 2023-01-14 11:56:00           Дмитрий     11:57:00   15:30:00   

     interval_from interval_to    duration device  \
0         11:56:57    12:35:42   38.750000     ПК   
1     

In [224]:
lag = 3
current.loc[(current['interval_from'] >= webinar_range[0]) &
            (current['interval_from'] <= webinar_range[1]) &
            (current['duration'] >= lag)].reset_index()['name'].nunique()


218